# Import Packages and Setup Environment

In [1]:
import numpy as np
import pandas as pd

In [2]:
#Allow printing of large numpy arrays.
# import sys
# import numpy
# numpy.set_printoptions(threshold=sys.maxsize)

In [3]:
# Remove max columns and row limit on pandas
pd.options.display.max_columns = None
pd.options.display.max_rows = 50000

# Read Position Data and Display for the Given Simulation Data

In [4]:
# Read in the binary numpy files storing the acceleration, velocity, position, and mass for 
# every body at every time step.
data_folder = "data/"

a_raw = np.load(data_folder + "a.npy")
v_raw = np.load(data_folder + "v.npy")
x_raw = np.load(data_folder + "x.npy")
m_raw = np.load(data_folder + "m.npy")

In [5]:
x_raw.shape

(2, 2, 2)

## Attempt to Convert Position Data to Pandas Multi-Index Dataframe

In [6]:
# Get the size of each dimension in numpy array.
# m-> the number of time steps in the simulation.
# n-> the number of bodies in the simulation.
# r-> number of dimensions in the vector holding the acceleration, displacement, etc.
pos_m,pos_n,pos_r = x_raw.shape
# Stack the XY or XYZ arrays of each body into columns, removing a dimension.
# np.column_stack() -> takes a sequence of 1D arrays and stacks them as columns in a 2D matrix.
# np.arange() provides evenly spaced values that repeat n times.  The new index.
# 
pos_arr = np.column_stack(
    (np.repeat(np.arange(pos_m),pos_n), 
     np.tile(np.arange(0,pos_n,1),pos_m), 
     x_raw.reshape(pos_m*pos_n,-1))
)
# Create dataframe from stacked column array.
pos_df = pd.DataFrame(pos_arr)
# Use df.groupby() to group by time step or planent and create MultiIndex for easy data referencing.
pos_df = pos_df.groupby([0,1]).mean()
pos_df.index.names = ['time_step', 'body']
pos_df.columns = ['pos_x', 'pos_y']
pos_df.head(20)

pos_x    pos_y
time_step body                       
0.0       0.0   1.496000e+11      0.0
          1.0   0.000000e+00      0.0
1.0       0.0  -6.637591e+19  29800.0
          1.0   1.993613e+14      0.0

In [7]:
pos_df

pos_x    pos_y
time_step body                       
0.0       0.0   1.496000e+11      0.0
          1.0   0.000000e+00      0.0
1.0       0.0  -6.637591e+19  29800.0
          1.0   1.993613e+14      0.0

## Attempt to Display Position Data on Bokeh Scatter Plot

In [8]:
# Bokeh and plotting related imports
# Plotting Imports
import bokeh.io
bokeh.io.output_notebook()  # Set plot output to embed in notebook.
import bokeh.layouts
import bokeh.plotting
# Other imports for multi-plot figures.
from bokeh.io import output_file, show
from bokeh.layouts import column
from bokeh.plotting import figure

Loading BokehJS ...

Need to create lists of numpy arrays that contain each body's position over time. \
List of arrays containing the positions of all bodies going back in time. = [bod1(x1, x2, x3, etc),(),()]

In [9]:
# Example of navigating Pandas MultiIndex to get X,Y position data.
pos_df.loc[0,1]

pos_x    0.0
pos_y    0.0
Name: (0.0, 1.0), dtype: float64

In [10]:
# For shits and giggles.  Seeing how many indexing levels there are currently.
pos_df.index.nlevels

2

How to index a Pandas dataframe with a multi-index.
https://pandas.pydata.org/pandas-docs/stable/user_guide/advanced.html
Great Stackoverflow summary on MultiIndex. https://stackoverflow.com/questions/53927460/select-rows-in-pandas-multiindex-dataframe

In [11]:
# Indexing to a specific column after navigating through the multiindex.
pos_df.loc[(0,1), 'pos_x']

0.0

In [12]:
# Use slicing to get all time steps for each body.
idx = pd.IndexSlice  # Need to create something called an indexer to kind of wrap the slice() function and make slicing easier.
test_slice = pos_df.loc[idx[0:10, 0], 'pos_x']
test_slice

time_step  body
0.0        0.0     1.496000e+11
1.0        0.0    -6.637591e+19
Name: pos_x, dtype: float64

In [13]:
# Shits and giggles.  Testing conversion of slice to numpy array.
test_slice.values

array([ 1.49600000e+11, -6.63759134e+19])

The above was mostly getting used to using MultiIndex Pandas dataframes.  Now trying to create the list of arrays that will be used for plotting paths of each body.

In [14]:
# Indexer to help with slicing dataframe.
idx = pd.IndexSlice
# List of bodies for each dimension whose elements are numpy arrays going back in time.  
# Each element is a time series for that body and that body's dimension.
pos_x_list = []
pos_y_list = []
# Group by level 1 to get the index of every body in the system.  Level 0 is the time step index.
# Index will represent the body number we are currently extracting the position time series for.
# Use slicing to get the position values for all time steps in a specified dimensions and save
# to the list of numpy arrays for that dimension.
for index, data in pos_df.groupby(level=1):
    # Get slice of data to create time series of position values
    # Use .to_numpy() to convert to numpy array
    temp_x_time_series = pos_df.loc[idx[:, index], 'pos_x'].to_numpy()
    temp_y_time_series = pos_df.loc[idx[:, index], 'pos_y'].to_numpy()
    # Add time series to respective dimension.
    pos_x_list.append(temp_x_time_series)
    pos_y_list.append(temp_y_time_series)
    
# At this point, we now have a list of bodies for each position dimension that contains a time series 
# for that dimension and each body.

### Plotting

Create functions to plot these lists of body time series

In [15]:
from bokeh.palettes import Turbo256 as palette
import itertools
from random import randint

def plot_2D_body_time_series(pos_x_list, pos_y_list, plot_width, plot_height, title):
    """
    Accepts lists for x and y dimensions whose elements are time series data and whose
    index represents the number of the body in the simulation.
    
    returns Bokeh figure to plot.
    """
    # Create Bokeh figure to add plots to
    f = bokeh.plotting.figure(
        title = title,
        plot_width = plot_width,
        plot_height = plot_height
    )

    # Generate line for each body.
    # Randomly select color from palette using randint
    for i in range(0,len(pos_x_list)):
        f.line(
            pos_x_list[i],
            pos_y_list[i],
            line_width = 1,
            color = palette[randint(0,255)],
            legend_label = str(i)
        )
    f.legend.location = 'top_left'
    return f

In [16]:
# Display body path plot
fig = plot_2D_body_time_series(
    pos_x_list = pos_x_list,
    pos_y_list = pos_y_list,
    plot_width = 800,
    plot_height = 800,
    title = "Body Paths Over Time"
)

bokeh.plotting.show(fig)

# Calculate Displacement for n Time Steps on Each Body

Set the number of time steps in the future to calculate displacement for.  This will add columns to the end of the displacement dataframe that will later be filled.

In [17]:
# Set the number of "shotgun" future time steps to predict.
num_ts_to_predict = 10

In [18]:
# Make copy of position dataframe and add columns for displacment.
dis_df = pos_df.copy(deep=True)
# Add columns to end of displacement dataframe using for loop.
for i in range(1,num_ts_to_predict+1):
    col_title_x = "dis_x_" + str(i)
    col_title_y = "dis_y_" + str(i)
    dis_df[col_title_x] = None
    dis_df[col_title_y] = None

# Drop the previous position columns.
dis_df.drop(['pos_x', 'pos_y'], axis=1, inplace=True)
dis_df.head(15)

dis_x_1 dis_y_1 dis_x_2 dis_y_2 dis_x_3 dis_y_3 dis_x_4  \
time_step body                                                           
0.0       0.0     None    None    None    None    None    None    None   
          1.0     None    None    None    None    None    None    None   
1.0       0.0     None    None    None    None    None    None    None   
          1.0     None    None    None    None    None    None    None   

               dis_y_4 dis_x_5 dis_y_5 dis_x_6 dis_y_6 dis_x_7 dis_y_7  \
time_step body                                                           
0.0       0.0     None    None    None    None    None    None    None   
          1.0     None    None    None    None    None    None    None   
1.0       0.0     None    None    None    None    None    None    None   
          1.0     None    None    None    None    None    None    None   

               dis_x_8 dis_y_8 dis_x_9 dis_y_9 dis_x_10 dis_y_10  
time_step body                                                    
0.0       0.0     None    None    None    None     None     None  
          1.0     None    None    None    None     None     None  
1.0       0.0     None    None    None    None     None     None  
          1.0     None    None    None    None     None     None

Will now loop over all time steps and future predicted time steps to calculate all displacements.

In [19]:
# Indexer to help with slicing dataframe.
idx = pd.IndexSlice
# Loop over all the time steps
for curr_time_step in dis_df.index.levels[0]:
    # Loop over the number of time steps in the future to be calculating
    # displacement for.
    # Don't do last time steps for displacement dataframe.  Can only look so many
    # time steps into the future before running out of data.
    if curr_time_step < (max(dis_df.index.levels[0]) - num_ts_to_predict):
        for num_ts_in_future in range(1, num_ts_to_predict + 1):
            col_title_x = "dis_x_" + str(num_ts_in_future)
            col_title_y = "dis_y_" + str(num_ts_in_future)
            dis_df.loc[idx[curr_time_step, :], [col_title_x, col_title_y]] = (
                pos_df.loc[curr_time_step + num_ts_in_future] 
                - pos_df.loc[curr_time_step]
            ).values

# Drop the time steps that could not be used for calculating displacecment.
# Create list of time steps to drop.
beg_drop_index = max(dis_df.index.levels[0]) - num_ts_to_predict
end_drop_index = max(dis_df.index.levels[0]) + 1
drop_list = list(range(int(beg_drop_index), int(end_drop_index)))
# Drop the time steps from the displacement dataframe.
dis_df.drop(drop_list, level=0, inplace=True)
# Ouput the new dataframe
dis_df.head(15)

,,dis_x_1,dis_y_1,dis_x_2,dis_y_2,dis_x_3,dis_y_3,dis_x_4,dis_y_4,dis_x_5,dis_y_5,dis_x_6,dis_y_6,dis_x_7,dis_y_7,dis_x_8,dis_y_8,dis_x_9,dis_y_9,dis_x_10,dis_y_10
time_step,body,,,,,,,,,,,,,,,,,,,,


# Aggregate the Velocity Data

Need to aggregate the velocity data so we know what the velocity of each body was at each time step and what it should be at the future, predicted time steps.

In [20]:
v_raw

array([[[0.00000000e+00, 2.98000000e+04],
        [0.00000000e+00, 0.00000000e+00]],

       [[0.00000000e+00, 8.94374443e-02],
        [0.00000000e+00, 8.95046082e-02]]])

In [21]:
# Construct the velocity dataframe from the raw simulator velocity output data.
# Get the size of each dimension in numpy array.
# m-> the number of time steps in the simulation.
# n-> the number of bodies in the simulation.
# r-> number of dimensions in the vector holding the acceleration, displacement, etc.
vel_m,vel_n,vel_r = v_raw.shape
# Stack the XY or XYZ arrays of each body into columns, removing a dimension.
# np.column_stack() -> takes a sequence of 1D arrays and stacks them as columns in a 2D matrix.
# np.arange() provides evenly spaced values that repeat n times.  The new index.
# 
vel_arr = np.column_stack(
    (np.repeat(np.arange(vel_m),vel_n), 
     np.tile(np.arange(0,vel_n,1),vel_m), 
     v_raw.reshape(vel_m*vel_n,-1))
)
# Create dataframe from stacked column array.
vel_df = pd.DataFrame(vel_arr)
# Use df.groupby() to group by time step or planent and create MultiIndex for easy data referencing.
vel_df = vel_df.groupby([0,1]).mean()
vel_df.index.names = ['time_step', 'body']
vel_df.columns = ['vel_x', 'vel_y']
vel_df.head(20)

vel_x         vel_y
time_step body                     
0.0       0.0     0.0  29800.000000
          1.0     0.0      0.000000
1.0       0.0     0.0      0.089437
          1.0     0.0      0.089505

Now need to add columns that will predict what the velocities will be in the future.

In [22]:
# Add columns to end of velocity dataframe using for loop.
for i in range(1,num_ts_to_predict+1):
    col_title_x = "vel_x_" + str(i)
    col_title_y = "vel_y_" + str(i)
    vel_df[col_title_x] = None
    vel_df[col_title_y] = None
# Keeping velocities at respective time steps in place.
vel_df.head(15)

vel_x         vel_y vel_x_1 vel_y_1 vel_x_2 vel_y_2 vel_x_3  \
time_step body                                                                
0.0       0.0     0.0  29800.000000    None    None    None    None    None   
          1.0     0.0      0.000000    None    None    None    None    None   
1.0       0.0     0.0      0.089437    None    None    None    None    None   
          1.0     0.0      0.089505    None    None    None    None    None   

               vel_y_3 vel_x_4 vel_y_4 vel_x_5 vel_y_5 vel_x_6 vel_y_6  \
time_step body                                                           
0.0       0.0     None    None    None    None    None    None    None   
          1.0     None    None    None    None    None    None    None   
1.0       0.0     None    None    None    None    None    None    None   
          1.0     None    None    None    None    None    None    None   

               vel_x_7 vel_y_7 vel_x_8 vel_y_8 vel_x_9 vel_y_9 vel_x_10  \
time_step body                                                            
0.0       0.0     None    None    None    None    None    None     None   
          1.0     None    None    None    None    None    None     None   
1.0       0.0     None    None    None    None    None    None     None   
          1.0     None    None    None    None    None    None     None   

               vel_y_10  
time_step body           
0.0       0.0      None  
          1.0      None  
1.0       0.0      None  
          1.0      None

Now we can grab velocities from future time steps and add them to the new columns.

In [23]:
# Indexer to help with slicing dataframe.
idx = pd.IndexSlice
# Loop over all the time steps
for curr_time_step in vel_df.index.levels[0]:
    # Loop over the number of time steps in the future to be grabbing
    # velocities from.
    # Don't do last time steps for displacement dataframe.  Can only look so many
    # time steps into the future before running out of data.
    if curr_time_step < (max(vel_df.index.levels[0]) - num_ts_to_predict):
        for num_ts_in_future in range(1, num_ts_to_predict + 1):
            col_title_x = "vel_x_" + str(num_ts_in_future)
            col_title_y = "vel_y_" + str(num_ts_in_future)
            vel_df.loc[idx[curr_time_step, :], [col_title_x, col_title_y]] = (
                vel_df.loc[idx[curr_time_step + num_ts_in_future, :], ['vel_x','vel_y']]
            ).values

# Drop the time steps that could not be used for calculating displacecment.
# Create list of time steps to drop.
beg_drop_index = max(vel_df.index.levels[0]) - num_ts_to_predict
end_drop_index = max(vel_df.index.levels[0]) + 1
drop_list = list(range(int(beg_drop_index), int(end_drop_index)))
# Drop the time steps from the displacement dataframe.
vel_df.drop(drop_list, level=0, inplace=True)
# Ouput the new dataframe
vel_df.head(15)

,,vel_x,vel_y,vel_x_1,vel_y_1,vel_x_2,vel_y_2,vel_x_3,vel_y_3,vel_x_4,vel_y_4,vel_x_5,vel_y_5,vel_x_6,vel_y_6,vel_x_7,vel_y_7,vel_x_8,vel_y_8,vel_x_9,vel_y_9,vel_x_10,vel_y_10
time_step,body,,,,,,,,,,,,,,,,,,,,,,


At this point, we have aggregated all the velocities for the future time steps we want to predict.

# Read In and Reformat Acceleration Data

Need to read in acceleration data to dataframe format so it is compatible for merging with all the other displacement, velocity, and mass data.

In [24]:
a_raw

array([[[ 1.32751827e+20, -5.95998211e+04],
        [-3.98722682e+14,  1.79009216e-01]]])

In [25]:
# Construct the acceleration dataframe from the raw simulator velocity output data.
# Get the size of each dimension in numpy array.
# m-> the number of time steps in the simulation.
# n-> the number of bodies in the simulation.
# r-> number of dimensions in the vector holding the acceleration, displacement, etc.
acc_m,acc_n,acc_r = a_raw.shape
# Stack the XY or XYZ arrays of each body into columns, removing a dimension.
# np.column_stack() -> takes a sequence of 1D arrays and stacks them as columns in a 2D matrix.
# np.arange() provides evenly spaced values that repeat n times.  The new index.
# Had to add 1 to arange to go from 1 to 79 instead of 0.
acc_arr = np.column_stack(
    (np.repeat(np.arange(1, acc_m+1, 1),acc_n), 
     np.tile(np.arange(0,acc_n,1),acc_m), 
     a_raw.reshape(acc_m*acc_n,-1))
)
# Create dataframe from stacked column array.
acc_df = pd.DataFrame(acc_arr)
# Use df.groupby() to group by time step or planent and create MultiIndex for easy data referencing.
acc_df = acc_df.groupby([0,1]).mean()
acc_df.index.names = ['time_step', 'body']
acc_df.columns = ['acc_x', 'acc_y']
acc_df.head(15)

acc_x         acc_y
time_step body                            
1.0       0.0   1.327518e+20 -59599.821125
          1.0  -3.987227e+14      0.179009

Need to drop time steps that could not be used in previous dataframes since they went beyond the ability to calculate displacement for the time steps we want to predict. \

Need to keep in mind that there is no acceleration at time step 0.  Acceleration only exists after 

In [26]:
# Drop the time steps that could not be used for calculating displacecment.
# Create list of time steps to drop.
beg_drop_index = max(acc_df.index.levels[0]) - num_ts_to_predict
end_drop_index = max(acc_df.index.levels[0]) + 1
drop_list = list(range(int(beg_drop_index), int(end_drop_index)))
# Drop the time steps from the displacement dataframe.
acc_df.drop(drop_list, level=0, inplace=True)
# Ouput the new dataframe
acc_df.head(15)

,,acc_x,acc_y
time_step,body,,


# Read In and Reformat Mass Data

Mass data needs to be reformatted to fit the MultiIndex of the acceleration, velocity, and displacement dataframes.

In [27]:
m_raw = m_raw.reshape(m_raw.shape[0])
m_raw.shape

(2,)

In [28]:
m_raw

array([5.974e+24, 1.989e+30])

Will need to repeat array of body masses by the number of time steps and create the 

In [29]:
# Construct the mass dataframe from the raw simulator mass output data.
# Get the size of each dimension in numpy array.
# m-> the number of time steps in the simulation.
# n-> the number of bodies in the simulation.
# r-> number of dimensions in the vector holding the acceleration, displacement, etc.
mass_n = m_raw.shape[0]
# Stack the Mass column with the indexing columns.  Mass column will be repeated by the number
# of time steps.
# np.column_stack() -> takes a sequence of 1D arrays and stacks them as columns in a 2D matrix.
# np.arange() provides evenly spaced values that repeat n times.  The new index.
# Using displacement dataframe dimensions to copy the masses enough.
mass_arr = np.column_stack(
     (np.repeat(np.arange(0, vel_m, 1),mass_n), 
     np.tile(np.arange(0,mass_n,1), vel_m),
     np.tile(m_raw, vel_m))
)
# Create dataframe from stacked column array.
mass_df = pd.DataFrame(mass_arr)
# Use df.groupby() to group by time step or planent and create MultiIndex for easy data referencing.
mass_df = mass_df.groupby([0,1]).mean()
mass_df.index.names = ['time_step', 'body']
mass_df.columns = ['mass']
mass_df.head(15)

mass
time_step body              
0.0       0.0   5.974000e+24
          1.0   1.989000e+30
1.0       0.0   5.974000e+24
          1.0   1.989000e+30

Remove the time steps not able to be calculated in displacement and velocity dataframes.

In [30]:
# Drop the time steps that could not be used for calculating displacecment.
# Create list of time steps to drop.
beg_drop_index = max(mass_df.index.levels[0]) - num_ts_to_predict
end_drop_index = max(mass_df.index.levels[0]) + 1
drop_list = list(range(int(beg_drop_index), int(end_drop_index)))
# Drop the time steps from the displacement dataframe.
mass_df.drop(drop_list, level=0, inplace=True)
# Ouput the new dataframe
mass_df.head(15)

,,mass
time_step,body,


# Merge Mass, Acceleration, Velocity, and Displacement Data

Start with the mass and merge it with the acceleration dataframe.

In [31]:
merged_data = mass_df.copy(deep=True)
merged_data = pd.merge(merged_data, acc_df, left_index=True, right_index=True, how='outer')
merged_data.head(15)

,,mass,acc_x,acc_y
time_step,body,,,


Merge in the velocity data.

In [32]:
merged_data = pd.merge(merged_data, vel_df, left_index=True, right_index=True, how='outer')
merged_data.head(15)

,,mass,acc_x,acc_y,vel_x,vel_y,vel_x_1,vel_y_1,vel_x_2,vel_y_2,vel_x_3,vel_y_3,vel_x_4,vel_y_4,vel_x_5,vel_y_5,vel_x_6,vel_y_6,vel_x_7,vel_y_7,vel_x_8,vel_y_8,vel_x_9,vel_y_9,vel_x_10,vel_y_10
time_step,body,,,,,,,,,,,,,,,,,,,,,,,,,


Merge in the displacement data.

In [33]:
merged_data = pd.merge(merged_data, dis_df, left_index=True, right_index=True, how='outer')
merged_data.head(15)

,,mass,acc_x,acc_y,vel_x,vel_y,vel_x_1,vel_y_1,vel_x_2,vel_y_2,vel_x_3,vel_y_3,vel_x_4,vel_y_4,vel_x_5,vel_y_5,vel_x_6,vel_y_6,vel_x_7,vel_y_7,vel_x_8,vel_y_8,vel_x_9,vel_y_9,vel_x_10,vel_y_10,dis_x_1,dis_y_1,dis_x_2,dis_y_2,dis_x_3,dis_y_3,dis_x_4,dis_y_4,dis_x_5,dis_y_5,dis_x_6,dis_y_6,dis_x_7,dis_y_7,dis_x_8,dis_y_8,dis_x_9,dis_y_9,dis_x_10,dis_y_10
time_step,body,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,


Now we will rearrange columns to make the format easier to understand.

In [34]:
# Create list of what the column order should be.
cols = []
cols.extend(['mass', 'acc_x', 'acc_y', 'vel_x', 'vel_y'])
# Loop over all the time steps we wanted to predict and rearrange the columns
# accordingly
for i in range(1,num_ts_to_predict+1):
    cols.append('dis_x_' + str(i))
    cols.append('dis_y_' + str(i))
    cols.append('vel_x_' + str(i))
    cols.append('vel_y_' + str(i))
# Rearrange columns using the create columns list.
merged_data = merged_data[cols]
# Drop the first time step for which there is no acceleration data.
merged_data.drop(0, level=0, inplace=True)
merged_data.head(15)

KeyError: 'labels [0] not found in level'

# Create Body Time Series Data Format

Create dataframe that has the time series associated with each body.  Should be able to just swap indexes around.

In [ ]:
merged_data_time_series = merged_data.copy(deep=True)

In [ ]:
merged_data_time_series = merged_data_time_series.swaplevel('time_step', 'body').sort_index(level=0)
merged_data_time_series.head(15)

# Attempt Converting Merged Datas to Numpy Arrays and Save as Both Pd dataframes and Np Arrays

In [ ]:
merged_data.to_numpy().shape

In [ ]:
dim0 = len(merged_data.index.get_level_values(0).unique())
dim1 = len(merged_data.index.get_level_values(1).unique())
dim2 = merged_data.shape[1]
merged_data_ndarray = merged_data.to_numpy().reshape((dim0, dim1, dim2))
merged_data_ndarray[0,0]

In [ ]:
dim0 = len(merged_data_time_series.index.get_level_values(0).unique())
dim1 = len(merged_data_time_series.index.get_level_values(1).unique())
dim2 = merged_data.shape[1]
merged_data_time_series_ndarray = merged_data_time_series.to_numpy().reshape((dim0, dim1, dim2))
merged_data_time_series_ndarray[0,5]

## Save the Numpy Arrays and Pandas Dataframes

In [ ]:
# Set the output directory.
out_dir = 'output/'

Save the dataframes by pickling them.

In [ ]:
merged_data.to_pickle(out_dir + 'sim_data_df-ts-body.pkl')
merged_data_time_series.to_pickle(out_dir + 'sim_data_df-body-ts.pkl')

Save the dataframes to XLSX files to view in Excel

In [ ]:
merged_data.to_excel(out_dir + 'sim_data_df-ts-body.xlsx')
merged_data_time_series.to_excel(out_dir + 'sim_data_df-body-ts.xlsx')

Save the numpy arrays by using numpy's saving function.

In [ ]:
np.save(out_dir + 'sim_data_np-ts-body.npy', merged_data_ndarray)
np.save(out_dir + 'sim_data_np-body-ts.npy', merged_data_time_series_ndarray)